In [ ]:
# required libraries

from google.colab import drive
drive.mount('/content/drive/')

from PIL import Image
import numpy as np
import os
import glob

from sklearn.model_selection import train_test_split

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm import tqdm


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# 각 섹션별 이미지가 담겨 있는 폴더의 경로들을 가져옴

section_folders = glob.glob('/content/drive/MyDrive/NaverNews/*')
section_folders

['/content/drive/MyDrive/NaverNews/economy',
 '/content/drive/MyDrive/NaverNews/world',
 '/content/drive/MyDrive/NaverNews/society',
 '/content/drive/MyDrive/NaverNews/science',
 '/content/drive/MyDrive/NaverNews/politics',
 '/content/drive/MyDrive/NaverNews/life']

In [ ]:
# 각 섹션별 사용할 데이터의 인덱스가 담겨 있는 딕셔너리

idx_dictionary = {
    'politics' : range(200),
    'society' : range(200),
    'science' : range(200),
    'life' : range(200),
    'world' : range(200),
    'economy' : range(200)
}

In [ ]:
# 분석에 사용할 모든 데이터들의 경로를 불러온다.

overall_imgpaths = []

for section_folder in section_folders:

  # 각 섹션 이름 가져와서 인덱스랑 합하기
  section = os.path.basename(section_folder)

  section_imgpaths = []

  for idx in idx_dictionary[section]:
    imgname = section + str(idx) + '.jpg'
    imgpath = os.path.join(section_folder, imgname)

    section_imgpaths.append(imgpath)

  overall_imgpaths.extend(section_imgpaths)

overall_imgpaths[:10]

['/content/drive/MyDrive/NaverNews/economy/economy0.jpg',
 '/content/drive/MyDrive/NaverNews/economy/economy1.jpg',
 '/content/drive/MyDrive/NaverNews/economy/economy2.jpg',
 '/content/drive/MyDrive/NaverNews/economy/economy3.jpg',
 '/content/drive/MyDrive/NaverNews/economy/economy4.jpg',
 '/content/drive/MyDrive/NaverNews/economy/economy5.jpg',
 '/content/drive/MyDrive/NaverNews/economy/economy6.jpg',
 '/content/drive/MyDrive/NaverNews/economy/economy7.jpg',
 '/content/drive/MyDrive/NaverNews/economy/economy8.jpg',
 '/content/drive/MyDrive/NaverNews/economy/economy9.jpg']

In [ ]:
# 전체 데이터의 라벨 (6개 섹션)

overall_labels = [imgpath.split('/')[-2] for idx, imgpath in enumerate(overall_imgpaths)]
overall_labels[:10]

['economy',
 'economy',
 'economy',
 'economy',
 'economy',
 'economy',
 'economy',
 'economy',
 'economy',
 'economy']

In [ ]:
root = '/content/drive/MyDrive/NaverNews'
sections = os.listdir(root)
sections

['economy', 'world', 'society', 'science', 'politics', 'life']

In [ ]:
# 라벨에 해당하는 y값 매칭하기 (0 ~ 5)

# label_to_y = {section : idx for idx, section in enumerate(sections)}
# label_to_y

{'economy': 0,
 'world': 1,
 'society': 2,
 'science': 3,
 'politics': 4,
 'life': 5}

In [ ]:
label_to_y = {
    'politics': 0,
    'society': 1,
    'science': 2,
    'life': 3,
    'world': 4,
    'economy': 5
}
label_to_y

{'politics': 0,
 'society': 1,
 'science': 2,
 'life': 3,
 'world': 4,
 'economy': 5}

In [ ]:
overall_labels

In [ ]:
# 전체 데이터의 y값 생성하기 (0 ~ 5)

overall_y = [label_to_y[overall_labels[idx]] for idx, label in enumerate(overall_labels)]
overall_y[:10]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [ ]:
# train / test set split -> 나중에 stratified K-fold cv 라이브러리로 대체

train_imgpaths, test_imgpaths, train_y, test_y = train_test_split(overall_imgpaths, overall_y, test_size = 0.2)

np.shape(train_imgpaths), np.shape(test_imgpaths)

((960,), (240,))

In [ ]:
# 이미지 데이터셋 만들기

class RoBaMFImageDataset(Dataset):
  def __init__(self, imgpaths, y, transform = None):
    super(RoBaMFImageDataset, self).__init__()

    self.imgpaths = imgpaths
    self.y = y
    self.transform = transform


  def __len__(self):

    return len(self.imgpaths)


  def __getitem__(self, idx):
    imgpath = self.imgpaths[idx]

    img = Image.open(imgpath).convert('RGB')
    img = self.transform(img)

    target = self.y[idx]

    return img, target

In [ ]:
# 사용할 모형에 맞게 이미지를 transform & normalize

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Resize(256),
    #transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Resize(256),
    #transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# 훈련데이터셋, 시험데이터셋 완성

train_dataset = RoBaMFImageDataset(train_imgpaths, train_y, train_transform)
test_dataset = RoBaMFImageDataset(test_imgpaths, test_y, test_transform)

In [ ]:
# 위 데이터셋을 모형 적합에 활용할 수 있도록 loader에 올림

train_loader = DataLoader(train_dataset, batch_size = 128, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 128, shuffle = True)

In [ ]:
# 정상적으로 로딩되었는지 확인

images, labels = next(iter(train_loader))
images[0].shape, labels[0]

(torch.Size([3, 224, 224]), tensor(3))

In [ ]:
# 모형의 가중치 업데이트를 위한 모듈

def model_train(model, data_loader, loss_fn, optimizer, device):
  model.train()
  size = len(data_loader.dataset)

  progress_bar = tqdm(data_loader)

  corr = 0
  running_loss = 0

  for X, y in progress_bar:
    X, y = X.to(device), y.to(device)

    # 예측하고 크로스엔트로피 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 그래티언트 초기화
    optimizer.zero_grad()

    # 역전파 알고리즘에 의한 그래디언트 계산
    loss.backward()

    # 그래디언트를 이용한 업데이트
    optimizer.step()

    # accuracy 계산을 위한 정답 개수 계산
    corr += (pred.argmax(1) == y).type(torch.float).sum().item()

    # 평균 크로스엔트로피 계산을 위한 합
    running_loss += loss.item() * X.size(0)

  # accuracy
  accuracy = corr / size
  running_loss = running_loss / size

  return accuracy, running_loss

In [ ]:
# 모형 평가를 위한 모듈

def model_evaluate(model, data_loader, loss_fn, device):
  size = len(data_loader.dataset)
  model.eval()

  with torch.no_grad():
    corr = 0
    running_loss = 0

    for X, y in data_loader:
      X, y = X.to(device), y.to(device)


      # 예측 확률 계산
      pred = model(X)
      loss = loss_fn(pred, y)

      # accuracy 계산을 위한 정답 개수 계산
      corr += (pred.argmax(1) == y).type(torch.float).sum().item()

      # 평균 크로스엔트로피 계산을 위한 합
      running_loss += loss.item() * X.size(0)

  # accuracy
  accuracy = corr / size
  running_loss = running_loss / size

  return accuracy, running_loss

In [ ]:
# pretrained MobileNetV2 불러오기

mobilenetv2 = models.mobilenet_v2(weights = 'DEFAULT')

In [ ]:
# 전이된 가중치는 고정

for i, (name, param) in enumerate(mobilenetv2.named_parameters()):
  param.requires_grad = False

  if i == 155:
    print('end')
    break

end


In [ ]:
class ImageModel(nn.Module):
  def __init__(self, mobilenetv2):
    super(ImageModel, self).__init__()

    self.mobilenetv2 = mobilenetv2

    self.fc = nn.Sequential(
              nn.Linear(1000, 1024),
              nn.ReLU(),
              nn.MaxPool1d(kernel_size=2, stride=2, padding=0),
              nn.Linear(512, 512),
              nn.ReLU(),
              nn.MaxPool1d(kernel_size=2, stride=2, padding=0),
              nn.Linear(256, 6),
              nn.Softmax(1)
              )


  def forward(self, img):
    x = self.mobilenetv2(img)
    x = self.fc(x)

    return x

In [ ]:
model = ImageModel(mobilenetv2)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr = 0.02)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', patience = 3, factor = 0.3)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#lr=0.2, patence=3, factor=0.3/ adamw, rmsprop/ lr한번 조절해보자!/loss fuction도..? 0.25앵간했음
#RBGA로 안나타나는 png파일을
#from PIL import Image

#image = Image.open('path_to_your_image.png')
#image = image.convert('RGBA')
#위 방식처럼 rgba방식으로 표현가능

In [ ]:
# 모형 훈련 및 검증
max_epoch = 1

for epoch in range(max_epoch):

  # 각 에포크별 모형 훈련 -> train accuracy와 손실함수 반환
  train_accuracy, train_loss = model_train(model, train_loader, loss_fn, optimizer, device)

  # 시험 데이터에 모형 적합 -> valiation accuray와 손실함수 반환
  val_accuracy, val_loss = model_evaluate(model, test_loader, loss_fn, device)

  scheduler.step(train_loss)

  print(f'''epoch {epoch + 1:02d} -------------------------------------------------- \n
          train_accuracy: {train_accuracy:.5f}, train_loss: {train_loss:.5f}, val_accuracy: {val_accuracy:.5f}, val_loss: {val_loss:.5f} \n\n''')

100%|██████████| 8/8 [01:59<00:00, 14.93s/it]


epoch 01 -------------------------------------------------- 

          train_accuracy: 0.14688, train_loss: 1.79201, val_accuracy: 0.15000, val_loss: 1.79231 




In [ ]:
torch.save(model.state_dict(),'/content/drive/My Drive/MobileNetV2.pth')